#LSTM

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

Loading the Dataset

In [2]:
df = pd.read_csv('Small.csv', index_col=False)
texts = df['Tweet']
labels = df['Sarcasm'].map({'yes': 1, 'no': 0})

In [3]:
max_len = 100
vocab_size = 10000
embedding_dim = 128

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

padded_sequences = pad_sequences(sequences, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

Function of prediction

In [4]:
def predict_sarcasm(text, tokenizer, model, max_len):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded_sequence)
    if prediction > 0.5:
        return "This text is predicted to be sarcastic."
    else:
        return "This text is predicted to be non-sarcastic."

Training the model

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
new_text = "My name is Tanisha."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)


Epoch 1/10
174/174 [==============================] - 37s 114ms/step - loss: 0.4406 - accuracy: 0.8081 - val_loss: 0.2665 - val_accuracy: 0.8997
Epoch 2/10
174/174 [==============================] - 12s 71ms/step - loss: 0.2264 - accuracy: 0.9134 - val_loss: 0.3096 - val_accuracy: 0.8911
Epoch 3/10
174/174 [==============================] - 11s 66ms/step - loss: 0.1476 - accuracy: 0.9484 - val_loss: 0.3130 - val_accuracy: 0.8882
Epoch 4/10
174/174 [==============================] - 11s 62ms/step - loss: 0.0662 - accuracy: 0.9760 - val_loss: 0.3709 - val_accuracy: 0.8759
Epoch 5/10
174/174 [==============================] - 11s 64ms/step - loss: 0.0586 - accuracy: 0.9811 - val_loss: 0.3518 - val_accuracy: 0.8947
Epoch 6/10
44/44 [==============================] - 1s 25ms/step - loss: 0.4989 - accuracy: 0.8925
Test Loss: 0.4989, Accuracy: 0.8925
1/1 [==============================] - 2s 2s/step
This text is predicted to be non-sarcastic.


Testing the model

In [6]:
new_text = "My mom asked me this question as well."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 33ms/step
This text is predicted to be non-sarcastic.


In [8]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 34ms/step
This text is predicted to be sarcastic.


Checking whether the model is giving 2 answers for one statement

In [9]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 32ms/step
This text is predicted to be sarcastic.
